# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-6emv5r2y
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-6emv5r2y
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit b93ccb75fa0b3117aaccbed55713d30aa4c63a36
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [5]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [6]:
model = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# len 2

In [ ]:
example_prompt = "3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 12.88 Prob: 18.99% Token: | 5|

Top 0th token. Logit: 12.88 Prob: 18.99% Token: | 5|
Top 1th token. Logit: 12.16 Prob:  9.20% Token: |/|
Top 2th token. Logit: 11.87 Prob:  6.89% Token: | 4|
Top 3th token. Logit: 11.56 Prob:  5.06% Token: |.|
Top 4th token. Logit: 11.29 Prob:  3.85% Token: | 3|
Top 5th token. Logit: 11.09 Prob:  3.16% Token: | 1|
Top 6th token. Logit: 11.06 Prob:  3.05% Token: |
|
Top 7th token. Logit: 10.99 Prob:  2.85% Token: | 6|
Top 8th token. Logit: 10.70 Prob:  2.14% Token: |-|
Top 9th token. Logit: 10.56 Prob:  1.86% Token: | 0|


Ranks of the answer tokens: [(' 5', 0)]

In [ ]:
example_prompt = "33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '33', ' 34']
Tokenized answer: [' 35']


Performance on answer token:
Rank: 4        Logit: 10.84 Prob:  1.87% Token: | 35|

Top 0th token. Logit: 12.30 Prob:  8.00% Token: |.|
Top 1th token. Logit: 11.43 Prob:  3.35% Token: |
|
Top 2th token. Logit: 11.26 Prob:  2.83% Token: | 34|
Top 3th token. Logit: 10.87 Prob:  1.92% Token: | 33|
Top 4th token. Logit: 10.84 Prob:  1.87% Token: | 35|
Top 5th token. Logit: 10.80 Prob:  1.78% Token: | 36|
Top 6th token. Logit: 10.60 Prob:  1.46% Token: | 0|
Top 7th token. Logit: 10.52 Prob:  1.34% Token: | 32|
Top 8th token. Logit: 10.47 Prob:  1.28% Token: | 29|
Top 9th token. Logit: 10.36 Prob:  1.15% Token: | 38|


Ranks of the answer tokens: [(' 35', 4)]

In [ ]:
example_prompt = "null 33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'null', ' 33', ' 34']
Tokenized answer: [' 35']


Performance on answer token:
Rank: 0        Logit: 16.19 Prob: 68.71% Token: | 35|

Top 0th token. Logit: 16.19 Prob: 68.71% Token: | 35|
Top 1th token. Logit: 12.85 Prob:  2.45% Token: | 33|
Top 2th token. Logit: 12.59 Prob:  1.89% Token: |
|
Top 3th token. Logit: 12.57 Prob:  1.85% Token: | 37|
Top 4th token. Logit: 12.45 Prob:  1.64% Token: | 34|
Top 5th token. Logit: 12.30 Prob:  1.41% Token: | 36|
Top 6th token. Logit: 12.10 Prob:  1.15% Token: | 40|
Top 7th token. Logit: 12.00 Prob:  1.04% Token: | 45|
Top 8th token. Logit: 11.98 Prob:  1.03% Token: | 0|
Top 9th token. Logit: 11.57 Prob:  0.68% Token: | 30|


Ranks of the answer tokens: [(' 35', 0)]

In [ ]:
example_prompt = "32 33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '32', ' 33', ' 34']
Tokenized answer: [' 35']


Performance on answer token:
Rank: 0        Logit: 17.90 Prob: 76.47% Token: | 35|

Top 0th token. Logit: 17.90 Prob: 76.47% Token: | 35|
Top 1th token. Logit: 14.62 Prob:  2.88% Token: |
|
Top 2th token. Logit: 14.52 Prob:  2.60% Token: | 36|
Top 3th token. Logit: 14.29 Prob:  2.07% Token: | 34|
Top 4th token. Logit: 13.95 Prob:  1.47% Token: | 37|
Top 5th token. Logit: 13.75 Prob:  1.20% Token: | 33|
Top 6th token. Logit: 13.50 Prob:  0.94% Token: | 40|
Top 7th token. Logit: 13.42 Prob:  0.87% Token: | 45|
Top 8th token. Logit: 12.86 Prob:  0.49% Token: | 30|
Top 9th token. Logit: 12.67 Prob:  0.41% Token: | 25|


Ranks of the answer tokens: [(' 35', 0)]

In [ ]:
example_prompt = "1 2"
example_answer = " 3"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2']
Tokenized answer: [' 3']


Performance on answer token:
Rank: 0        Logit: 16.69 Prob: 89.34% Token: | 3|

Top 0th token. Logit: 16.69 Prob: 89.34% Token: | 3|
Top 1th token. Logit: 12.56 Prob:  1.44% Token: |
|
Top 2th token. Logit: 11.64 Prob:  0.58% Token: | 4|
Top 3th token. Logit: 11.10 Prob:  0.33% Token: | 1|
Top 4th token. Logit: 10.85 Prob:  0.26% Token: | 5|
Top 5th token. Logit: 10.80 Prob:  0.25% Token: | Next|
Top 6th token. Logit: 10.78 Prob:  0.24% Token: |.|
Top 7th token. Logit: 10.35 Prob:  0.16% Token: | 2|
Top 8th token. Logit: 10.16 Prob:  0.13% Token: | /|
Top 9th token. Logit: 10.11 Prob:  0.12% Token: |/|


Ranks of the answer tokens: [(' 3', 0)]

In [ ]:
example_prompt = "March April"
example_answer = " May"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'March', ' April']
Tokenized answer: [' May']


Performance on answer token:
Rank: 0        Logit: 22.59 Prob: 96.57% Token: | May|

Top 0th token. Logit: 22.59 Prob: 96.57% Token: | May|
Top 1th token. Logit: 17.70 Prob:  0.73% Token: | July|
Top 2th token. Logit: 17.46 Prob:  0.57% Token: | June|
Top 3th token. Logit: 16.94 Prob:  0.34% Token: | April|
Top 4th token. Logit: 16.85 Prob:  0.31% Token: | September|
Top 5th token. Logit: 16.68 Prob:  0.26% Token: | 5|
Top 6th token. Logit: 16.60 Prob:  0.24% Token: |
|
Top 7th token. Logit: 16.34 Prob:  0.19% Token: | November|
Top 8th token. Logit: 16.20 Prob:  0.16% Token: | March|
Top 9th token. Logit: 15.67 Prob:  0.10% Token: | October|


Ranks of the answer tokens: [(' May', 0)]

# add 2

In [ ]:
example_prompt = "2 4 6 8 10 12 14 18"
example_answer = " 20"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12', ' 14', ' 18']
Tokenized answer: [' 20']


Performance on answer token:
Rank: 0        Logit: 17.63 Prob: 36.05% Token: | 20|

Top 0th token. Logit: 17.63 Prob: 36.05% Token: | 20|
Top 1th token. Logit: 16.80 Prob: 15.70% Token: | 19|
Top 2th token. Logit: 16.54 Prob: 12.17% Token: | 22|
Top 3th token. Logit: 16.44 Prob: 11.02% Token: | 21|
Top 4th token. Logit: 15.81 Prob:  5.85% Token: | 24|
Top 5th token. Logit: 15.24 Prob:  3.33% Token: | 23|
Top 6th token. Logit: 15.14 Prob:  2.99% Token: | 18|
Top 7th token. Logit: 14.89 Prob:  2.33% Token: |
|
Top 8th token. Logit: 13.69 Prob:  0.70% Token: | 30|
Top 9th token. Logit: 13.59 Prob:  0.63% Token: | 25|


Ranks of the answer tokens: [(' 20', 0)]

In [7]:
example_prompt = "4 6 8 10 12 14 18 20 22"
example_answer = " 24"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '4', ' 6', ' 8', ' 10', ' 12', ' 14', ' 18', ' 20', ' 22']
Tokenized answer: [' 24']


Performance on answer token:
Rank: 0        Logit: 21.54 Prob: 60.97% Token: | 24|

Top 0th token. Logit: 21.54 Prob: 60.97% Token: | 24|
Top 1th token. Logit: 20.01 Prob: 13.21% Token: | 23|
Top 2th token. Logit: 19.71 Prob:  9.76% Token: | 25|
Top 3th token. Logit: 18.95 Prob:  4.61% Token: | 26|
Top 4th token. Logit: 18.22 Prob:  2.20% Token: | 27|
Top 5th token. Logit: 18.21 Prob:  2.20% Token: | 28|
Top 6th token. Logit: 17.82 Prob:  1.48% Token: | 29|
Top 7th token. Logit: 17.80 Prob:  1.45% Token: | 22|
Top 8th token. Logit: 17.47 Prob:  1.04% Token: | 30|
Top 9th token. Logit: 17.37 Prob:  0.94% Token: |
|


Ranks of the answer tokens: [(' 24', 0)]

In [8]:
example_prompt = "12 14 18 20 22"
example_answer = " 24"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '12', ' 14', ' 18', ' 20', ' 22']
Tokenized answer: [' 24']


Performance on answer token:
Rank: 1        Logit: 19.31 Prob: 24.06% Token: | 24|

Top 0th token. Logit: 19.32 Prob: 24.14% Token: | 23|
Top 1th token. Logit: 19.31 Prob: 24.06% Token: | 24|
Top 2th token. Logit: 18.64 Prob: 12.30% Token: | 25|
Top 3th token. Logit: 17.97 Prob:  6.28% Token: | 26|
Top 4th token. Logit: 17.67 Prob:  4.67% Token: | 27|
Top 5th token. Logit: 17.59 Prob:  4.29% Token: | 29|
Top 6th token. Logit: 17.11 Prob:  2.66% Token: |
|
Top 7th token. Logit: 17.11 Prob:  2.66% Token: | 22|
Top 8th token. Logit: 17.01 Prob:  2.42% Token: | 28|
Top 9th token. Logit: 16.75 Prob:  1.86% Token: | 30|


Ranks of the answer tokens: [(' 24', 1)]

In [11]:
example_prompt = "12 14 18 20 22 24 26 28 30"
example_answer = " 32"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '12', ' 14', ' 18', ' 20', ' 22', ' 24', ' 26', ' 28', ' 30']
Tokenized answer: [' 32']


Performance on answer token:
Rank: 1        Logit: 16.76 Prob: 17.40% Token: | 32|

Top 0th token. Logit: 16.76 Prob: 17.44% Token: | 31|
Top 1th token. Logit: 16.76 Prob: 17.40% Token: | 32|
Top 2th token. Logit: 16.63 Prob: 15.37% Token: | 33|
Top 3th token. Logit: 16.16 Prob:  9.63% Token: | 35|
Top 4th token. Logit: 15.71 Prob:  6.10% Token: | 34|
Top 5th token. Logit: 15.49 Prob:  4.91% Token: |
|
Top 6th token. Logit: 15.21 Prob:  3.69% Token: | 36|
Top 7th token. Logit: 14.80 Prob:  2.47% Token: | 30|
Top 8th token. Logit: 14.52 Prob:  1.86% Token: | 40|
Top 9th token. Logit: 14.10 Prob:  1.23% Token: | 3|


Ranks of the answer tokens: [(' 32', 1)]